In [1]:
!pip install ordered-set fpdf2 pdf2image  openpyxl

In [2]:
# @title IMPORTS

from ordered_set import OrderedSet
import time
import re
import os
import tempfile
import requests
import json
from fpdf import FPDF
from base64 import b64encode
from IPython.display import display, HTML
import os
import pandas as pd
from google.colab import drive
from datetime import datetime
from openpyxl import load_workbook
from calendar import monthcalendar
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side
from openpyxl.utils import get_column_letter
drive.mount('/content/drive')
import calendar
from datetime import datetime, timedelta


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# @title hidden APi calls

def get_food_orders_summary(date,email,password):



    # Login and get token
    login_url = "https://admin.nompenang.com/auth/login"
    payload = {"email": email, "password": password}

    session = requests.Session()
    response = session.post(login_url, json=payload)
    token = response.json()['data']['access_token']

    # Fetch orders
    orders_url = "https://admin.nompenang.com/items/orders"
    headers = {"Authorization": f"Bearer {token}"}
    params = {
        "fields": "id,order_number, customer_name, customer_phone, delivery_method, status, date_created, schedule_date, schedule_time, total_amount, payment_method, delivery_address",
        "limit": 500
    }

    response = session.get(orders_url, headers=headers, params=params)
    data = response.json()
    orders = data['data']

    Dates = [
    datetime.strptime(order['schedule_date'], '%Y-%m-%d')
    for order in orders
    ]
    target_date = datetime.strptime(date, '%Y-%m-%d')

    completed_order_numbers = [
        order for order in orders
        if order['schedule_date'] == date and order['status'] != 'cancelled'
    ]
    list_of_order_numbers = [order['id'] for order in completed_order_numbers]
    total_amount = sum(float(order['total_amount']) for order in completed_order_numbers)

    return total_amount



In [4]:


def get_weeks_of_month(year, month): # emulates monthcal, but returns a list of dicts
    """Get all weeks of the specified month"""
    cal = calendar.monthcalendar(year, month)
    weeks = []

    for week in cal:
        week_days = []
        for day in week:
            if day == 0:  # Day belongs to previous/next month
                week_days.append(None)
            else:

                date = datetime(year, month, day)
                formatted_date = date.strftime('%Y-%m-%d')
                day_name = calendar.day_name[date.weekday()]
                week_days.append({
                    'formatted_date': formatted_date,
                    'day_number': day,
                    'day_name': day_name,
                    'date_obj': date
                })
        weeks.append(week_days)
    return weeks



In [5]:
def workbook (year, month, weeks, email, password):
  workbook = load_workbook('/content/drive/MyDrive/Colab_Notebooks/Sales Data.xlsx')
  center_align = Alignment(horizontal='center', vertical='center') #Alignment for Excel
  full_month_name = calendar.month_name[month] # Covert int month to str Month
  sheet_name = f"{full_month_name} Online Sales" # Name of the Excel Worksheet
  dark_fill = PatternFill(start_color="366092",  # Dark blue
                       end_color="366092",
                       fill_type="solid")
  if sheet_name in workbook.sheetnames: # Select worksheet if exists

      ws = workbook[f"{full_month_name} Online Sales"]

  else:        # Else create new sheet

      ws = workbook.create_sheet(f"{full_month_name} Online Sales")



  ws['A1'] = f"{full_month_name}" #title for Table
  current_row = 2 # Start row for Headers
  current_week = 1 # Start of the month starts on Week 1
  days_of_the_week = ['Day of the Week', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
  currency_format = '$#,##0.00' #custom currency format for excel

  for col, header in enumerate(days_of_the_week, 1): #start column count at 1 because excel counts starting at 1 instead of 0

          cell = ws.cell(row=current_row, column=col, value = header)
          cell.alignment = center_align

  current_row += 1  # One row after header



  for i, week in enumerate(weeks, 1): # Loops for each week in the month and titles each row as that specific week

    week_cell = ws.cell(row=current_row, column=1)
    week_cell.alignment = center_align
    week_cell.font = Font(bold=True)
    week_cell.value = f"Week {current_week}"
    current_column = 2 # Sets the starting postion to Monday every week

    for day_info in week:

              cell = ws.cell(row = current_row, column = current_column) # create cell object so can be modified later
              if day_info != None: # If day info does exist get sales for the day

                  total_amount = get_food_orders_summary(day_info['formatted_date'], email, password)
                  cell.value = round((total_amount), 2) # Round up for currencies
                  cell.alignment = center_align  # Formatting
                  cell.number_format = currency_format # Formatting

              else:

                  cell.value = '' # blank value to signify days that are not in the month
                  cell.alignment = center_align # Formatting
                  cell.fill = dark_fill
              current_column+=1 # Everytime it looks it moves to the next column/day

    current_week += 1
    current_row += 1 # Everytime it loops it moves to the next row and week

  for col in range(2, 9):

      start_row = 3
      end_row = current_row - 1
      formula = f"=SUM({get_column_letter(col)}{start_row}:{get_column_letter(col)}{end_row})"
      cell = ws.cell(row=8, column=col, value = formula)
      cell.font = Font(bold=True)
      cell.alignment = center_align

  ws['I8'] = '=SUM(B3:H7)' #Grand total for month
  ws.sheet_format.defaultColWidth = 20 # Formatting for creation of new cells

  for col in range(1, ws.max_column + 1): # Formatting for already populated cells

      col_letter = get_column_letter(col)
      ws.column_dimensions[col_letter].width = 20

  ws.merge_cells('A1:H1') # Title
  ws['A1'].alignment = center_align  # Formatting
  ws['A1'].font = Font(bold=True, size=16, color="1F4E78") # Formatting
  ws['I8'].alignment = center_align
  ws['I8'].number_format = currency_format # Formatting

  for row in ws.iter_rows(): # Border Formatting

          for cell in row:

            cell.font = Font(size=13)
            cell.border = Border(left=Side(style='thin'),
                              right=Side(style='thin'),
                              top=Side(style='thin'),
                              bottom=Side(style='thin'))

            if cell.row > 8 or cell.column > 9 : #reset borders if cells are beyond max columns

              cell.border = Border()

  return workbook


In [7]:
year = 2025 # Year
month = 9 # Month in int
email = '#######' #redacted 
password ='#######' #redacted
weeks = get_weeks_of_month(year, month) # List of dicts for totals.
workbook = workbook(year , month, weeks, email, password)
workbook.save('/content/temp_sales_data.xlsx')
!cp '/content/temp_sales_data.xlsx' '/content/drive/MyDrive/Colab_Notebooks/Sales Data.xlsx' #bash in colab
print("File saved successfully!")

File saved successfully!
